In [1]:
import os

%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L

import sys
sys.path.append('../')

from squad_tools import load_dict
from os.path import join

ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/i258346/.local/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/home/i258346/.local/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/home/i258346/.local/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 60, in init_dev
    sched=config.gpuarray.sched)
  File "pygpu/gpuarray.pyx", line 634, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 584, in pygpu.gpuarray.pygpu_init
  File "pygpu/gpuarray.pyx", line 1057, in pygpu.gpuarray.GpuContext.__cinit__
GpuArrayException: cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [2]:
%aimport QANet

floatX == float32
device == cuda


In [3]:
glove_ver = '6B'

glove_path = join('/pio/data/data/glove_vec', glove_ver, 'glove')
glove_embs = np.load(join(glove_path, 'glove.' + glove_ver + '.300d.npy'))
voc_size = glove_embs.shape[0]

In [4]:
glove_words = load_dict(join(glove_path, '..', 'glove.' + glove_ver + '.300d.txt'))
NAW_token = glove_words.index('<not_a_word>')

# Weryfikacja modeli

In [5]:
squad_base_path = '/pio/data/data/squad/'
squad_path = join(squad_base_path, 'glove' + glove_ver)

In [7]:
from squad_tools import load_squad_train, filter_empty_answers, trim_data

train_data = load_squad_train(squad_path)
train_data = filter_empty_answers(train_data)
train_data = trim_data(train_data, 300)

Only positive samples.


In [6]:
from squad_tools import load_squad_dev

dev_data = load_squad_dev(squad_base_path, squad_path, lower_raw=glove_ver=='6B',
                          make_negative=True, NAW_token=NAW_token)

Adding NAW token to dev set.


In [13]:
from squad_tools import load_squad_dev

dev_data_pos = load_squad_dev(squad_base_path, squad_path, lower_raw=glove_ver=='6B',
                          make_negative=False, NAW_token=NAW_token)

In [24]:
%%time
qa_net = QANet.QANet(voc_size=voc_size,
                     dev_data=dev_data_pos,
                     emb_init=glove_embs,
                     skip_train_fn=True,
                     train_unk=True,
                     negative=False,
                     conv='valid')

Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 8.25 s, sys: 66.6 ms, total: 8.31 s
Wall time: 8.19 s


In [25]:
qa_net.load_params('../models/6B/best/model.ep09.npz')

In [26]:
%%time
qa_net.calc_dev_f1(0)

Calculating validation F1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
Getting answers...
Done
F1:  72.366287839
EM:  63.1409649953
CPU times: user 1min 4s, sys: 1.66 s, total: 1min 6s
Wall time: 1min 6s


72.36628783904861

In [ ]:
'''
best:
    Pos: F1: 71.47   Exact: 61.99
best_neg:
    Pos: F1: 71.07   Exact: 62.08
    Neg: F1: 60.31   Exact: 53.67
'''


'''
OLD

charemb_trainNAW_dropout_memory_crashed/charemb_trainNAW_dropout_ep5.npz
    Pos: F1: 68.12   Exact: 58.04
    Neg: F1: 60.17   Exact: 51.84
    
    Classification task:
        [[TN=9857  FP= 455]
         [FN=1275  TP=9295]]
    
charemb_glove_fixed_dropout_ep13.npz
    Pos: F1: 69.56   Exact: 59.33
'''